In [14]:
!pip install Augmentor

Defaulting to user installation because normal site-packages is not writeable


In [17]:
import Augmentor
import numpy as np
from PIL import Image
import random
import shutil

In [25]:
def list_train_test_split(data_dir):
    imgs = []
    for i in sorted(os.listdir(os.path.join(data_dir, "GT"))):
        img_path = os.path.join(data_dir, "GT", i)
        if os.path.isfile(img_path):
            imgs.append(img_path)

    train_split = 0.8
    train_samples = random.sample(imgs, int(0.8*len(imgs)))

    with open(os.path.join(data_dir, "test.txt"), "w") as f1:
        with open(os.path.join(data_dir, "train.txt"), "w") as f2:
            for s in imgs:
                if s in train_samples:
                    f2.write(s+"\n")
                else:
                    f1.write(s+"\n")

In [31]:
def move_to_train_test_dirs(trainfile, testfile):
    train, test = [], []
    with open(trainfile, "r") as f:
        for l in f.readlines():
            train.append(l.strip())
    
    with open(testfile, "r") as f:
        for l in f.readlines():
            test.append(l.strip())
    
    shutil.rmtree("ohaze/train")
    shutil.rmtree("ohaze/test")
    
    os.makedirs("ohaze/train/GT")
    os.makedirs("ohaze/test/GT")
    os.makedirs("ohaze/train/hazy")
    os.makedirs("ohaze/test/hazy")

    for tr in train:
        shutil.move(tr, f"ohaze/train/GT/{os.path.split(tr)[1]}")
        shutil.move(tr.replace("GT", "hazy"), f"ohaze/train/GT/{os.path.split(tr)[1]}".replace("GT", "hazy"))
    
    for tr in test:
        shutil.move(tr, f"ohaze/test/GT/{os.path.split(tr)[1]}")
        shutil.move(tr.replace("GT", "hazy"), f"ohaze/test/GT/{os.path.split(tr)[1]}".replace("GT", "hazy"))
    

In [37]:
def clean_names(data_dir):
    """Makes the file names in the hazy and GT directories 
    the same so that Augmentor detects corresponding files """
    for f in os.listdir(data_dir):
        if "hazy" in f:
            shutil.move(os.path.join(data_dir, f), os.path.join(data_dir, f.replace("_hazy", "")))
        elif "GT" in f:
            shutil.move(os.path.join(data_dir, f), os.path.join(data_dir, f.replace("_GT", "")))

In [33]:
list_train_test_split("ohaze")
move_to_train_test_dirs("ohaze/train.txt", "ohaze/test.txt")

In [38]:
clean_names("ohaze/train/hazy")
clean_names("ohaze/train/GT")
clean_names("ohaze/test/hazy")
clean_names("ohaze/test/GT")

In [39]:
p = Augmentor.Pipeline("ohaze/train/hazy")
p.ground_truth("ohaze/train/GT")

Initialised with 36 image(s) found.
Output directory set to ohaze/train/hazy/output.

36 ground truth image(s) found.


In [40]:
p.rotate(probability=0.5, max_left_rotation=10, max_right_rotation=10)
p.flip_left_right(probability=0.8)
p.crop_random(probability=0.5, percentage_area=0.85)
p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=8)

In [41]:
p.sample(1000)

Processing <PIL.Image.Image image mode=RGB size=1359x903 at 0x7FE1452007F0>: 100%|██████████| 1000/1000 [01:03<00:00, 15.74 Samples/s]                 
